In [ ]:
# !pip3 install absl-py
# !pip3 install --upgrade tensorflow
# !pip3 install "tensorflow>=2.0.0"
# !pip3 install --upgrade tensorflow-hub

# !pip3 install --ignore-installed Pillow==9.0.0

In [ ]:
import matplotlib.pyplot as plt


#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re  # RegEx
import seaborn as sns

import spacy
nlp = spacy.load('de_core_news_lg')

import xmltodict

In [ ]:
import time

In [ ]:
%%time
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" 
#@param ["https://tfhub.dev/google/universal-sentence-encoder/4", 
# "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder-large/5 loaded
CPU times: user 9.2 s, sys: 1.11 s, total: 10.3 s
Wall time: 11 s


In [ ]:
%%time
skills = pd.read_csv('./data/skills_de.csv',usecols=['conceptUri','preferredLabel','description'])

CPU times: user 72 ms, sys: 15.6 ms, total: 87.6 ms
Wall time: 89.4 ms


In [ ]:
skills

,conceptUri,preferredLabel,description
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,Musikpersonal verwalten,Zuweisen und Verwalten der Aufgaben des Person...
1,http://data.europa.eu/esco/skill/00064735-8fad...,Strafvollzugsverfahren beaufsichtigen,Überwachen des Betriebs einer Justizvollzugsan...
2,http://data.europa.eu/esco/skill/000709ed-2be5...,nicht unterdrückende Praktiken anwenden,"Ermitteln von Repressionen in Gesellschaften, ..."
3,http://data.europa.eu/esco/skill/0007bdc2-dd15...,Einhaltung von Vorschriften von Eisenbahnfahrz...,"Kontrollieren von Fahrzeugen, Komponenten und ..."
4,http://data.europa.eu/esco/skill/00090cc1-1f27...,verfügbare Dienste ermitteln,Ermitteln der verschiedenen verfügbaren Dienst...
...,...,...,...
13886,http://data.europa.eu/esco/skill/ffef5eb3-a15e...,berufliche Leistungsfähigkeit von Nutzern/Nutz...,"Wiederherstellen der kognitiven, sensomotorisc..."
13887,http://data.europa.eu/esco/skill/fff0b074-5a76...,Beleuchtung in Transportgeräten einbauen,Einbau von Beleuchtungselementen in Transportg...
13888,http://data.europa.eu/esco/skill/fff0e2cd-d0bd...,Verarbeitung natürlicher Sprache,"Technologien, die es IKT-Geräten ermöglichen, ..."
13889,http://data.europa.eu/esco/skill/fff5bc45-b506...,Bauarbeiten koordinieren,Koordinierung der Tätigkeiten mehrerer Bauarbe...


In [ ]:
labels = skills['preferredLabel'].copy(deep=True)
labels

0                                  Musikpersonal verwalten
1                    Strafvollzugsverfahren beaufsichtigen
2                  nicht unterdrückende Praktiken anwenden
3        Einhaltung von Vorschriften von Eisenbahnfahrz...
4                             verfügbare Dienste ermitteln
                               ...                        
13886    berufliche Leistungsfähigkeit von Nutzern/Nutz...
13887             Beleuchtung in Transportgeräten einbauen
13888                     Verarbeitung natürlicher Sprache
13889                             Bauarbeiten koordinieren
13890         Absturzsicherungen und Bordbretter anbringen
Name: preferredLabel, Length: 13891, dtype: object

In [ ]:
def labels_processing(text):
    processed = []
    for item in text:
        item = nlp(item)
        itemString = ''
        for word in item:
            word = word.lemma_.lower()
            if word != '--' and word != '' and word != ' ':
                if word == '\xa0': continue                    
                itemString += word + ' '
        processed.append(itemString[:-1])
    return processed

In [ ]:
%%time
processedLabels = labels_processing(labels)

CPU times: user 1min 18s, sys: 809 ms, total: 1min 19s
Wall time: 1min 21s


In [ ]:
pd.DataFrame(processedLabels, columns = ['processedLabel'])

,processedLabel
0,musikpersonal verwalten
1,strafvollzugsverfahr beaufsichtigen
2,nicht unterdrückend praktik anwenden
3,einhaltung von vorschrift von eisenbahnfahrzeu...
4,verfügbar dienst ermitteln
...,...
13886,beruflich leistungsfähigkeit von nutzer nutzer...
13887,beleuchtung in transportgerät einbauen
13888,verarbeitung natürlich sprache
13889,bauarbeit koordinieren


In [ ]:
%%time
xml_data = open('./data/FOKUS_AWT_CompetencyExtraction_WB_Brandenburg.xml','r', encoding='ISO-8859-15').read() # read data
xmlDict = xmltodict.parse(xml_data) # parse xml

CPU times: user 6.98 s, sys: 291 ms, total: 7.27 s
Wall time: 7.35 s


In [ ]:
%%time
xmlcourses = xmlDict['DEFTISCAT']['COURSETRANSACTIONS']['INSERTCOURSES']['COURSE']

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [ ]:
%%time
coursesDict = []
for course in xmlcourses:
    coursesDict.append({"course_name": course['CS_NAME'],
                 "course_id": course['CS_ID'], 
                 "course_description": course['CS_DESC_LONG']})
courses_table = pd.DataFrame(coursesDict)
courses = courses_table['course_name'] + ' ' + courses_table['course_description']

CPU times: user 45.8 ms, sys: 17.3 ms, total: 63.1 ms
Wall time: 65.1 ms


In [ ]:
courses_table

,course_name,course_id,course_description
0,"""Schwierige"" Klienten? - Mit Patienten, Angehö...",4A873264-7ADD-DE47-3039-1FDA692E8164,- Analyse strapaziöser Gesprächsmuster\n- Schw...
1,Aktuelles Arbeitsrecht 2022,99BEDCEB-4FF3-3F8F-88B7-B30E50638F01,Kurzbeschreibung\nDas Arbeitsrecht unterliegt ...
2,Ambulante Pflege - Rechtssicher Handeln und Ha...,97FD41FD-1A91-179C-3EA5-6A23E2F436D5,- Grundlagen der straf- und zivilrechtlichen H...
3,Aufgaben des gesetzlichen Betreuers - Zur Refo...,A0CC573A-79D7-79A4-5B22-F675C4F06950,Kurzbeschreibung\nNoch im Jahr 2020 plant der ...
4,Basisqualifikation für ungelernte Pflegekräfte...,7C449EF7-12A8-82FC-0E5E-BD2885FDB93C,- Alten- und Krankenpflege\n . Körperpflege\n...
...,...,...,...
16847,5 Monate Weiterbildung: Organisation & Führung...,A1E046BE-1C4B-B977-C9B9-88D8D2860107,Die aktuell vorherrschende Situation auf dem A...
16848,Conversion und Usability Experte,A1DF8EEB-D317-70BA-508F-AFC732369860,Ziel der Maßnahme ist es den Teilnehmern eine ...
16849,Digital Transformation Management,A1DD9D98-2BA3-3A11-0A24-72FC4C79422A,Ziel der Maßnahme ist es den Teilnehmern eine ...
16850,E-Commerce Geschäftsmodelle,A1DD60F7-9B6E-ECEE-F1DF-53FED2DCCC5C,Ziel der Maßnahme ist es den Teilnehmern eine ...


In [ ]:
courses

0        "Schwierige" Klienten? - Mit Patienten, Angehö...
1        Aktuelles Arbeitsrecht 2022 Kurzbeschreibung\n...
2        Ambulante Pflege - Rechtssicher Handeln und Ha...
3        Aufgaben des gesetzlichen Betreuers - Zur Refo...
4        Basisqualifikation für ungelernte Pflegekräfte...
                               ...                        
16847    5 Monate Weiterbildung: Organisation & Führung...
16848    Conversion und Usability Experte Ziel der Maßn...
16849    Digital Transformation Management Ziel der Maß...
16850    E-Commerce Geschäftsmodelle Ziel der Maßnahme ...
16851    Experte im Digital Content Creation Die Teilne...
Length: 16852, dtype: object

In [ ]:
nlp.add_pipe('merge_noun_chunks')

<function spacy.pipeline.functions.merge_noun_chunks(doc: spacy.tokens.doc.Doc) -> spacy.tokens.doc.Doc>

In [ ]:
print(nlp.pipe_names)

['tok2vec', 'tagger', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner', 'merge_noun_chunks']


In [ ]:
%%time
def Courses_processing(courses):
    allChunks = []
    courses = list(nlp.pipe(courses))
    for course in courses:
        chunks = []
        for chunk in course:
            with nlp.select_pipes(disable=['merge_noun_chunks','attribute_ruler', 'ner']):
                chunk = nlp(chunk.text)
                text = ''
                for word in chunk:
                    if not word.is_stop:
                        word = word.lemma_.lower().strip() 
                        word = word.strip('@#$%^&*§')
                        if word != '--' and word != '' and word != ' 'and word != '\n*' and word != '\n\n':
#                             if "/-" in word: word = word.split("/-")[0]
                            text += word + ' '
                if text != '':
                    chunks.append(text[:-1])
        allChunks.append(chunks)
    return allChunks

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 8.82 µs


In [ ]:
%%time
def To_sentences(courses):
    courses_sent = []
    courses_sent_lemma = []
    for course in (courses):
        doc = nlp(str(course))
        sentences = []
        for sent in doc.sents:
            course_processed = ''
            sents = []
            for word in sent:
                word = word.text
                word = word.strip().strip('"@#$%^&*§')
                if word == '-':
                    if course_processed != '':
                        sents.append(course_processed[:-1])
                        course_processed = ''
                    continue
                if (not re.match('\s+', word,re.I)) and word != '--' and word != '':
    #                 and word != ' ' 
                    if "/-" in word: 
                        word = word.split("/-")[0]
                    if re.match(r'\,|\.|\?|\!|\)', word):
                        course_processed = course_processed[:-1] + word + ' '
                    elif re.match(r'\(',word):
                        course_processed += word
                    else:
                        course_processed += word + ' '
            sentences += sents
            if course_processed != '':
                sentences.append(course_processed[:-1])
        courses_sent_lemma.append(sentences)
    return courses_sent_lemma

CPU times: user 8 µs, sys: 20 µs, total: 28 µs
Wall time: 14.1 µs


In [ ]:
%%time
course_in_sentences = To_sentences(courses)


CPU times: user 8min 14s, sys: 51.2 s, total: 9min 6s
Wall time: 9min 13s


[['Schwierige" Klienten?',
  'Mit Patienten, Angehörigen und Kollegen clever kommunizieren',
  'Analyse strapaziöser Gesprächsmuster',
  'Schwierige Klienten oder schwierige Situation?',
  'Was heißt es schwierige Gespräche souverän zu meistern?',
  '- Übungen mit Videofeedback werden durchgeführt!'],
 ['Aktuelles Arbeitsrecht 2022 Kurzbeschreibung',
  'Das Arbeitsrecht unterliegt einem ständigen Wandel durch Änderungen der Gesetzgebung und der folgenden Rechtsprechung.',
  'Im Seminar erhalten Sie notwendige Kenntnisse dieser Vorschriften bezogen auf soziale Einrichtungen, um rechtssicher zu handeln und unnötige gerichtliche Auseinandersetzungen zu vermeiden.',
  '- Beschäftigtendatenschutz nach EUDSGVO und BDSG',
  '- Urlaubsrecht, Urlaubsplanung, Verfall von Ansprüchen, Abgeltung von Urlaub',
  'Qualifizierungsvereinbarungen mit Bindungsklausel - Befristung von Arbeitsverträgen mit und ohne Sachgrund',
  '- Inhaltliche Gestaltungsmöglichkeiten bei Arbeitsverträgen',
  'Arbeitszeitre

In [ ]:
course_in_sentences

In [ ]:
%%time
courses_chunks = []

for course in course_in_sentences[:20]:
    
    course_chunks = []
    for sent in course:
        sent_chunks = []
        doc = nlp(sent)
        for tok in doc:
            with nlp.select_pipes(disable=['merge_noun_chunks','attribute_ruler', 'ner']):
                if not re.match('\(|\)|\,|\.|\!|\?|\/',tok.text) and not tok.is_stop: 
                    tok = nlp(tok.text)
                    words = ''
                    for word in tok:
                        if not word.is_stop and word.lemma_ != '--':
                            words = words + word.lemma_ + ' '  
                    sent_chunks.append(words[:-1])
        course_chunks += sent_chunks
    courses_chunks.append(course_chunks)


CPU times: user 3.39 s, sys: 22.6 ms, total: 3.41 s
Wall time: 3.44 s


In [ ]:
courses_chunks

[['schwierig Klient',
  'Patient',
  'angehörige',
  'Kollege',
  'clever',
  'kommunizieren',
  'Analyse',
  'strapaziös Gesprächsmuster',
  'schwierig Klient',
  'schwierig Situation',
  'schwierig Gespräch',
  'souverän',
  'meistern',
  '',
  'Übung',
  'Videofeedback',
  'durchführen'],
 ['Aktuelles Arbeitsrecht',
  '2022 Kurzbeschreibung',
  'Arbeitsrecht',
  'unterliegen',
  'ständig Wandel',
  'Änderung',
  'Gesetzgebung',
  'folgend Rechtsprechung',
  'Seminar',
  'erhalten',
  'notwendig Kenntnis',
  'Vorschrift',
  'beziehen',
  'sozial Einrichtung',
  'rechtssicher',
  'handeln',
  'unnötig gerichtlich Auseinandersetzung',
  'vermeiden',
  'Beschäftigtendatenschutz',
  'EUDSGVO',
  'BDSG',
  'Urlaubsrecht',
  'Urlaubsplanung',
  'Verfall',
  'Anspruch',
  'Abgeltung',
  'Urlaub',
  'Qualifizierungsvereinbarunge',
  'Bindungsklausel',
  '',
  'Befristung',
  'Arbeitsverträgen',
  'Sachgrund',
  'Inhaltliche Gestaltungsmöglichkeit',
  'Arbeitsverträgen',
  'Arbeitszeitrecht',

## Embedding-phase

In [ ]:
%%time
skills_embeddings = embed(processedLabels)
skills_embeddings

CPU times: user 1min 15s, sys: 26.7 s, total: 1min 41s
Wall time: 25.3 s


<tf.Tensor: shape=(13891, 512), dtype=float32, numpy=
array([[-0.09679928, -0.01319801,  0.02749727, ..., -0.00338633,
        -0.01980454,  0.00640224],
       [ 0.01513428,  0.01030495, -0.00539185, ...,  0.03585461,
        -0.03738291,  0.00346469],
       [ 0.0040679 ,  0.03890802,  0.07119194, ..., -0.03898589,
        -0.01094731,  0.00980166],
       ...,
       [ 0.01275541,  0.01799202,  0.03028508, ...,  0.01438965,
        -0.0098445 ,  0.05236192],
       [-0.03056339,  0.05143588,  0.02478526, ...,  0.03583746,
        -0.0386817 , -0.0045007 ],
       [-0.02028036,  0.04226601,  0.00409167, ..., -0.02154097,
         0.00466969,  0.04008627]], dtype=float32)>

In [ ]:
%%time
courses_chunks_embed = []
for course in courses_chunks:
    courses_chunks_embed.append(embed(course))

CPU times: user 3.98 s, sys: 2.54 s, total: 6.51 s
Wall time: 1.39 s


## Similarity

In [ ]:
%%time
def similarity(labels, courses):
    result = []
    for course in courses:
        result.append(np.dot(labels, np.transpose(course)))
    return result

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 11.9 µs


In [ ]:
%%time
simi_chunks_4 = similarity(skills_embeddings,courses_chunks_embed)

CPU times: user 570 ms, sys: 46 ms, total: 616 ms
Wall time: 159 ms


In [ ]:
%%time
idx = 0
counter = 0
for result in simi_chunks_4:
    print('counter: ', idx)
    print('max: ',np.max(result))
    c = np.where(result>0.785) 

    print(len(c[0]))
    for j in range(len(c[0])):
        print(processedLabels[c[0][j]],'--', c4[idx][c[1][j]])
    print(' ------')
    print(courses[idx])
    print('\n')
    idx += 1 
    counter += len(c[0])
print(counter)  

counter:  0
max:  0.8438107
1
schwierig situation bewältigen -- schwierig Situation
 ------
"Schwierige" Klienten? - Mit Patienten, Angehörigen und Kollegen clever kommunizieren - Analyse strapaziöser Gesprächsmuster
- Schwierige Klienten oder schwierige Situation?
- Was heißt es schwierige Gespräche souverän zu meistern?
- Übungen mit Videofeedback werden durchgeführt!


counter:  1
max:  0.9999998
60
wein lesen -- erhalten
arbeitsrecht -- Arbeitsrecht
schreibunterricht geben -- erhalten
erster hilfe leisten -- erhalten
schnell umschminken -- erhalten
schnell denken -- erhalten
unvoreingenommenheit zeigen -- erhalten
auf waldkrankheite kontrollieren -- erhalten
sicher medizinmanagement -- erhalten
sozial unternehmen -- sozial Einrichtung
montagezeichnung lesen -- erhalten
vortrag halten -- erhalten
karte lesen -- erhalten
bei rechtsstreitigkeit helfen -- erhalten
lösung für informationsfrage finden -- erhalten
finanziell unterstützung erhalten -- erhalten
tanzpartitur lesen -- erhalte